* вся работа делается в google colab, т.к. Spark работает только через него

# ШАГ 1. Развертываем Postgre сервер

In [2]:
# Установка PostgreSQL
!apt-get update
!apt-get install -y postgresql postgresql-contrib
!service postgresql start

# Установка клиента Python
!pip install psycopg2-binary pandas

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,218 B/s)
Reading package lists... Done
W: https://repo.mongodb.org/apt/ubuntu/dists/focal/mongodb-org/6.0/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources'

In [ ]:
# Настройка пароля и создание БД
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'postgres';"
!sudo -u postgres createdb ecommerce_db

import psycopg2
import pandas as pd
import csv

In [25]:
# Подключение к PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="ecommerce_db",
    user="postgres",
    password="postgres"
)
conn.autocommit = True
cur = conn.cursor()

In [26]:
# удаляем старые таблицы (потому что на то, чтобы заставить данные заливаться правильно, потребовалось несколько попыток)
cur.execute("DROP TABLE IF EXISTS product_analytics_monthly CASCADE")
cur.execute("DROP TABLE IF EXISTS order_items CASCADE")
cur.execute("DROP TABLE IF EXISTS orders CASCADE")
cur.execute("DROP TABLE IF EXISTS products CASCADE")

# создаем таблицы
cur.execute("""
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    category VARCHAR(100),
    price DECIMAL(10, 2)
);
""")

cur.execute("""
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date TIMESTAMP
);
""")

cur.execute("""
CREATE TABLE order_items (
    order_item_id INT,
    order_id INT,
    product_id INT,
    quantity INT,
    price DECIMAL(10, 2)
);
""")

cur.execute("""
CREATE TABLE product_analytics_monthly (
    product_id INT,
    total_quantity BIGINT,
    total_revenue DECIMAL(15, 2),
    order_count BIGINT,
    avg_rating DECIMAL(3, 2),
    positive_reviews BIGINT,
    negative_reviews BIGINT,
    total_reviews BIGINT,
    processing_date DATE
);
""")

# загружаем CSV в pandas (из которого потом будем заливать в postgre)
products_df = pd.read_csv('products.csv')
orders_df = pd.read_csv('orders.csv')
order_items_df = pd.read_csv('order_items.csv')

# products
for _, row in products_df.iterrows():
    cur.execute(
        "INSERT INTO products VALUES (%s, %s, %s, %s)",
        (int(row['product_id']), row['product_name'], row['category'], float(row['price']))
    )

# orders
for _, row in orders_df.iterrows():
    cur.execute(
        "INSERT INTO orders VALUES (%s, %s, %s)",
        (int(row['order_id']), int(row['customer_id']), row['order_date'])
    )

# order_items
for _, row in order_items_df.iterrows():
    cur.execute(
        "INSERT INTO order_items VALUES (%s, %s, %s, %s, %s)",
        (int(row['order_item_id']), int(row['order_id']), int(row['product_id']),
         int(row['quantity']), float(row['price']))
    )

In [28]:
cur.execute("SELECT COUNT(*) FROM orders")
print(f"Orders загружено: {cur.fetchone()[0]}")

cur.execute("SELECT COUNT(*) FROM order_items")
print(f"Order items загружено: {cur.fetchone()[0]}")

cur.execute("SELECT COUNT(*) FROM products")
print(f"Products загружено: {cur.fetchone()[0]}")

#cur.close()
#conn.close()

Orders загружено: 1000
Order items загружено: 2422
Products загружено: 50


# ШАГ 2. Развертываем MongoDB сервер

In [33]:
!curl -fsSL https://www.mongodb.org/static/pgp/server-6.0.asc | sudo apt-key add -
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!apt update
!apt install -y mongodb-org
!mkdir -p /data/db
!mongod --fork --logpath /var/log/mongod.log

!pip install pymongo pandas

OK
deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 3,917 B in 1s (4,193 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
6 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://repo.mongodb.org/

In [36]:
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS=5000)
client.server_info()
print("MongoDB успешно подключен")

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 5.0s, Topology Description: <TopologyDescription id: 6950397df78771f7990abdf2, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [30]:
# подключение
client = MongoClient('localhost', 27017)
db = client['ecommerce']
reviews = db['reviews']

# загрузка CSV
csv_df = pd.read_csv('data.reviews.csv')
reviews.insert_many(csv_df.to_dict('records'))

# загрузка JSON
with open('data.reviews.json', 'r') as f:
    json_data = [json.loads(line) for line in f if line.strip()]
    reviews.insert_many(json_data)

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6950374df78771f7990abdf0, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

Ушло много времени, чтобы попробовать подключить mongo, но, к сожалению, так и не получилось сделать так, чтобы он работал. Поскольку было разрешено, я воспользовалась возможностью залить все через postgres. Для чистоты эксперимента, было созданы две независимые БД

In [37]:
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'postgres';"
!sudo -u postgres createdb ecommerce_db2

conn1 = psycopg2.connect(
    host="localhost",
    database="ecommerce_db2",
    user="postgres",
    password="postgres"
)
conn1.autocommit = True
cur = conn1.cursor()

# создаем временную таблицу
cur.execute("""
CREATE TEMP TABLE temp_reviews (
    _id TEXT,
    review_id TEXT,
    product_id INT,
    customer_id INT,
    rating INT,
    created_at TEXT,
    helpful_votes INT
);
""")

# загружаем CSV во временную таблицу
with open('data.reviews.csv', 'r') as f:
    cur.copy_expert("COPY temp_reviews FROM STDIN WITH CSV HEADER", f)

# создаем основную таблицу
cur.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    _id VARCHAR(50),
    review_id VARCHAR(100) PRIMARY KEY,
    product_id INT,
    customer_id INT,
    rating INT,
    created_at TIMESTAMP,
    helpful_votes INT
);
""")

# копируем из временной в основную
cur.execute("""
INSERT INTO reviews
SELECT _id, review_id, product_id, customer_id, rating,
       created_at::TIMESTAMP, helpful_votes
FROM temp_reviews
ON CONFLICT (review_id) DO NOTHING
""")

cur.execute("SELECT COUNT(*) FROM reviews")
print(f"Отзывов загружено: {cur.fetchone()[0]}")

cur.close()
conn1.close()

ALTER ROLE
Отзывов загружено: 700


# ШАГ3: Работаем со Spark

In [38]:
# установка Spark
!pip install pyspark==3.5.0

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425346 sha256=a6be1ec8d93d7408e48740606f04155118071bc71a1f6636d701ed386462fc7a
  Stored in directory: /root/.cache/pip/wheels/84/40/20/65eefe766118e0a8f8e385cc3ed6e9eb7241c7e51cfc04c51a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.1
    Uninstalling pyspark-4.0.1:
      Successfully uninstalled pyspark-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [40]:
# создаем Spark сессию
spark = SparkSession.builder \
    .appName("EcommerceAnalytics") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.0") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

In [47]:
# даты для фильтрации
from datetime import datetime

# фиксируем период августа 2025 года (это последний месяц в табличках с заказами, за который что-то есть, посмотрела фильтрацией по дате в эксельке)
start_date = "2025-08-01"
end_date = "2025-08-31"

print(f"Загружаем данные за август 2025: {start_date} - {end_date}")

Загружаем данные за август 2025: 2025-08-01 - 2025-08-31


In [50]:
# параметры подключения
db_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

# заказы за последний месяц (из ecommerce_db)
orders_query = f"""
(SELECT * FROM orders
 WHERE order_date >= '{start_date}'
 AND order_date <= '{end_date}') as orders_last_month
"""

orders_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ecommerce_db") \
    .option("dbtable", orders_query) \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .load()

In [51]:
print(f"Заказов за месяц: {orders_df.count()}")

Заказов за месяц: 60


In [52]:
# pагрузка позиций заказов -

order_ids = [row.order_id for row in orders_df.select("order_id").collect()]

if order_ids:
    order_ids_str = ",".join(map(str, order_ids))
    order_items_query = f"""
    (SELECT * FROM order_items
     WHERE order_id IN ({order_ids_str})) as order_items_august
    """

    order_items_df = spark.read \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://localhost/ecommerce_db") \
        .option("dbtable", order_items_query) \
        .option("user", db_properties["user"]) \
        .option("password", db_properties["password"]) \
        .option("driver", db_properties["driver"]) \
        .load()

In [53]:
print(f"Загружено позиций заказов: {order_items_df.count()}")

Загружено позиций заказов: 153


In [54]:
# загрузка отзывов (из ecommerce_db2)
reviews_query = f"""
(SELECT * FROM reviews
 WHERE created_at >= '{start_date}'
 AND created_at <= '{end_date}') as reviews_august
"""

reviews_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ecommerce_db2") \
    .option("dbtable", reviews_query) \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .load()

In [56]:
print(f"Загружено отзывов: {reviews_df.count()}")

Загружено отзывов: 73


Задание 1: сгруппируйте по product_id и посчитайте количество уникальных заказов, общее количество проданных единиц и выручку.

In [57]:
# объединяем orders и order_items
transactions_df = orders_df.join(
        order_items_df,
        "order_id",
        "inner"
    )

from pyspark.sql.functions import countDistinct, sum, col

postgres_stats_df = transactions_df.groupBy("product_id").agg(
        countDistinct("order_id").alias("order_count"),
        sum("quantity").alias("total_quantity"),
        sum(col("quantity") * col("price")).alias("total_revenue")
    )

postgres_stats_df.show()

+----------+-----------+--------------+-------------+
|product_id|order_count|total_quantity|total_revenue|
+----------+-----------+--------------+-------------+
|        31|          3|             6|     35384.20|
|        34|          3|             8|     68828.58|
|        28|          3|             7|     27667.79|
|        27|          5|             8|     50893.16|
|        26|          6|            10|     19604.30|
|        44|          2|             3|     17299.30|
|        12|          5|            10|     50926.23|
|        47|          6|            12|     59750.96|
|         1|          1|             1|      4995.12|
|        13|          2|             3|       774.49|
|        16|          2|             3|     17601.62|
|         6|          1|             2|      6515.22|
|         3|          4|             8|     11040.75|
|        40|          2|             5|      2793.35|
|        20|          5|            10|     24768.03|
|        48|          5|    

Задание 2: отфильтруйте отзывы по оценке, сгруппируйте по product_id и посчитайте средний рейтинг, общее количество отзывов, количество положительных и негативных отзывов.

In [59]:
from pyspark.sql.functions import avg, count, when

reviews_stats_df = reviews_df.groupBy("product_id").agg(
        avg("rating").alias("avg_rating"),
        count("*").alias("total_reviews"),
        # Положительные отзывы (оценка 4-5)
        sum(when((col("rating") >= 4) & (col("rating") <= 5), 1).otherwise(0)).alias("positive_reviews"),
        # Негативные отзывы (оценка 1-2)
        sum(when((col("rating") >= 1) & (col("rating") <= 2), 1).otherwise(0)).alias("negative_reviews")
    )

reviews_stats_df.show()

+----------+------------------+-------------+----------------+----------------+
|product_id|        avg_rating|total_reviews|positive_reviews|negative_reviews|
+----------+------------------+-------------+----------------+----------------+
|        31|               4.0|            2|               1|               0|
|        34|               5.0|            1|               1|               0|
|        26|               2.0|            1|               0|               1|
|        44|              1.75|            4|               0|               3|
|        12|2.6666666666666665|            3|               1|               2|
|        47|               4.0|            3|               2|               1|
|         1|               3.0|            1|               0|               0|
|        13|               1.0|            1|               0|               1|
|         6|               2.5|            2|               1|               1|
|        16|               1.0|         

Задание 3: объедините два полученных DataFrame по полю product_id.

In [60]:
from pyspark.sql.functions import lit

combined_df = postgres_stats_df.join(
    reviews_stats_df,
    "product_id",
    "outer"
)

combined_df = combined_df.fillna({
    "order_count": 0,
    "total_quantity": 0,
    "total_revenue": 0.0,
    "avg_rating": 0.0,
    "total_reviews": 0,
    "positive_reviews": 0,
    "negative_reviews": 0
})

print(f"Всего продуктов: {combined_df.count()}")
combined_df.show(truncate=False)


Всего продуктов: 49
+----------+-----------+--------------+-------------+------------------+-------------+----------------+----------------+
|product_id|order_count|total_quantity|total_revenue|avg_rating        |total_reviews|positive_reviews|negative_reviews|
+----------+-----------+--------------+-------------+------------------+-------------+----------------+----------------+
|1         |1          |1             |4995.12      |3.0               |1            |0               |0               |
|2         |3          |7             |25802.10     |1.6666666666666667|3            |0               |3               |
|3         |4          |8             |11040.75     |2.0               |2            |0               |2               |
|4         |1          |2             |10655.60     |3.0               |3            |1               |1               |
|5         |4          |5             |48561.21     |1.0               |1            |0               |1               |
|6         |

Задание 4: Запишите новые рассчитанные данные в таблицу product_analytics_monthly используя JDBC-коннектор.

In [62]:
from pyspark.sql.functions import current_date

final_with_date_df = combined_df.withColumn(
    "processing_date",
    current_date().cast("date")
)

# параметры подключения (в уже знакомую базу ecommerce_db)
db_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

final_with_date_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ecommerce_db") \
    .option("dbtable", "product_analytics_monthly") \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .mode("append") \
    .save()

In [66]:
# финальный чек, делаем селект из постгре и смотрим, что все получилось

conn = psycopg2.connect(
    host="localhost",
    database="ecommerce_db",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()
query = """
SELECT
    *
FROM product_analytics_monthly
ORDER BY total_revenue DESC
LIMIT 5
"""

df = pd.read_sql_query(query, conn)
print(df)

   product_id  total_quantity  total_revenue  order_count  avg_rating  \
0          18              18      147857.95            8         2.0   
1          33              14      121602.21            6         2.0   
2          50              11       82378.58            5         3.0   
3          37              11       81389.44            5         2.5   
4          34               8       68828.58            3         5.0   

   positive_reviews  negative_reviews  total_reviews processing_date  
0                 0                 1              1      2025-12-27  
1                 0                 2              2      2025-12-27  
2                 1                 1              2      2025-12-27  
3                 0                 1              2      2025-12-27  
4                 1                 0              1      2025-12-27  


/tmp/ipython-input-3922193342.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
